In [22]:
! pip install -U osmnx jupyter selenium webdriver-manager scikit-learn matplotlib

  Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.16.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
! sudo apt install netconvert

In [1]:
import osmnx as ox
import networkx as nx

# --- Configure osmnx ---
ox.settings.all_oneway = True

# --- Download the graph for Berkeley ---
place_name = "Berkeley, California, USA"
print(f"Downloading graph for {place_name}...")
G = ox.graph_from_place(place_name, network_type='drive')


# --- FIX THE DATA: Impute missing edge speeds and calculate travel times ---

# 1. Impute missing speed limits based on road type
# This is the crucial step that fixes the error.
G = ox.add_edge_speeds(G)

# 2. Now, calculate travel time for every edge. This will now work.
G = ox.add_edge_travel_times(G)


print("\nGraph processed. Speeds and travel times have been added to each road segment.")
print("--- Example Edge Data ---")

# Let's inspect a single edge to see our new attributes
# Note: it might not have an original 'maxspeed' but will now have 'speed_kph' and 'travel_time'
edge_example = list(G.edges(data=True))[0]
print(edge_example)


# --- Find and plot the quickest route ---
origin_point = (37.8716, -122.2727) # Near Downtown Berkeley
destination_point = (37.8546, -122.2524) # Near Claremont Canyon

# Get the nearest network nodes to our points
origin_node = ox.nearest_nodes(G, origin_point[1], origin_point[0])
destination_node = ox.nearest_nodes(G, destination_point[1], destination_point[0])

print(f"\nRouting from node {origin_node} to {destination_node}.")

# Find the Shortest Path using our 'travel_time'
# The 'weight' parameter tells NetworkX to use our calculated travel_time
shortest_path_nodes = nx.shortest_path(G, source=origin_node, target=destination_node, weight='travel_time')

# Calculate and Print the Total Trip Time
total_trip_time_seconds = nx.shortest_path_length(G, source=origin_node, target=destination_node, weight='travel_time')
print(f"Total estimated trip time: {total_trip_time_seconds / 60:.2f} minutes")

# Visualize the Route
fig, ax = ox.plot_graph_route(G, shortest_path_nodes, route_color='cyan', node_size=0)


Graph processed. Speeds and travel times have been added to each road segment.
--- Example Edge Data ---
(35719098, 35719102, {'osmid': [5149917, 877720646, 435440767], 'highway': 'residential', 'maxspeed': '25 mph', 'name': 'Hillside Avenue', 'reversed': False, 'length': np.float64(204.83960829789945), 'bridge': 'yes', 'lanes': '1', 'geometry': <LINESTRING (-122.248 37.866, -122.248 37.866, -122.248 37.866, -122.248 37....>, 'speed_kph': 40.2335, 'travel_time': 18.328571709456995})

Routing from node 53116963 to 221406324.


NetworkXNoPath: No path between 53116963 and 221406324.

In [13]:
! netconvert --osm-files map.osm.xml -o map.net.xml

Success.


In [17]:
import osmnx as ox
import networkx as nx

# (You can reuse the graph 'G' from the previous script if you are in the same session)

# --- Find origin and destination nodes ---
# Let's pick two points on the map to route between
origin_point = (37.8716, -122.2727) # Near Downtown Berkeley
destination_point = (37.8546, -122.2524) # Near Claremont Canyon

# Get the nearest network nodes to our points
origin_node = ox.nearest_nodes(G, origin_point[1], origin_point[0])
destination_node = ox.nearest_nodes(G, destination_point[1], destination_point[0])

print(f"Routing from node {origin_node} to {destination_node}.")

# --- Find the Shortest Path using our 'travel_time' ---
# The 'weight' parameter tells NetworkX to use our calculated travel_time
# as the cost for each edge.
shortest_path_nodes = nx.shortest_path(G, source=origin_node, target=destination_node, weight='travel_time')

# --- Calculate and Print the Total Trip Time ---
total_trip_time_seconds = nx.shortest_path_length(G, source=origin_node, target=destination_node, weight='travel_time')
print(f"\nTotal estimated trip time: {total_trip_time_seconds / 60:.2f} minutes")

# --- Visualize the Route ---
# The 'route' parameter highlights the list of nodes we found.
fig, ax = ox.plot_graph_route(G, shortest_path_nodes, route_color='cyan', node_size=0)

NameError: name 'G' is not defined